In [ ]:
import numpy as np
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import torch
import torch.nn as nn
import dtnnlib as dtnn
# import resnet_cifar

In [ ]:
import scipy.stats as st

## Image Dot Product

In [ ]:
N = 5
W = np.ones((N,N))

In [ ]:
def gkern(kernlen=21, nsig=3):
    """Returns a 2D Gaussian kernel."""

    x = np.linspace(-nsig, nsig, kernlen+1)
    kern1d = np.diff(st.norm.cdf(x))
    kern2d = np.outer(kern1d, kern1d)
    return kern2d/kern2d.sum()

In [ ]:
G = gkern(N, 2.5)
G = G/G.max()

In [ ]:
G

In [ ]:
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(3, 5))
ax = ax0
ax.imshow(G, vmin=0, vmax=1, cmap="gray")
for i in range(G.shape[0]):
    for j in range(G.shape[1]):
        v = float(G[i, j])
        text = ax.text(j, i, f"{v:.2f}",
                       ha="center", va="center", color=["white", "k"][v>0.5])
ax.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)

ax = ax1
ax.imshow(W, vmin=0, vmax=1, cmap="gray")
color = "orange"
for i in range(W.shape[0]):
    for j in range(W.shape[1]):
        v = float(W[i, j])
        text = ax.text(j, i, f"{v:.2f}",
                       ha="center", va="center", color=["white", "k"][v>0.5])
    if i >= W.shape[0]-1:
        continue
    ax.hlines(i+0.5, -0.5, N-0.5, color="k")
    ax.vlines(i+0.5, -0.5, N-0.5, color="k")
    pass

plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)

# plt.show()

In [ ]:
alphas = np.linspace(0, 1, num=10)

## For dot products

In [ ]:
### From zeros to G
dot_prods0 = []
for alpha in alphas:
    A = alpha*G #+ (1-alpha)*W
    dp = (A*G).sum() ### calculating the dot product
#     dp = dp/np.linalg.norm(A)
    dot_prods0.append(dp)

In [ ]:
dot_prods0

In [ ]:
### From G to ones
dot_prods1 = []
for alpha in alphas:
    A = alpha*W + (1-alpha)*G
    dp = (A*G).sum() ### calculating the dot product    
#     dp = dp/np.linalg.norm(A)
    dot_prods1.append(dp)

In [ ]:
dot_prods1

In [ ]:
plt.plot(alphas, dot_prods0, marker='.')
plt.plot(alphas+1, dot_prods1, marker='.')

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.ticker import FormatStrFormatter

In [ ]:
! mkdir outputs/01_image_dot_product

In [ ]:
## Plot the output with images

fig, ax = plt.subplots(1, 1, figsize=(6,5))

plt.plot(alphas, dot_prods0, marker='.')
plt.plot(alphas+1, dot_prods1, marker='.')

for idx_alpha, alpha in enumerate(alphas):
    if idx_alpha+1 == len(alphas):
        continue
    A = alpha*G #+ (1-alpha)*W
    dp = (A*G).sum() ### calculating the dot product
    
    im = OffsetImage(A, norm=matplotlib.colors.Normalize(vmin=0, vmax=1), cmap="gray", zoom=3)
    ab = AnnotationBbox(im, (alpha, dp+0.5), frameon=True, pad=0)
    ax.add_artist(ab)
    
for alpha in alphas:
    A = alpha*W + (1-alpha)*G
    dp = (A*G).sum() ### calculating the dot product    
    im = OffsetImage(A, norm=matplotlib.colors.Normalize(vmin=0, vmax=1), cmap="gray", zoom=3)
    ab = AnnotationBbox(im, (1+alpha, dp+0.5), frameon=True, pad=0)
    ax.add_artist(ab)

ax.scatter(alphas[-1], dot_prods0[-1], marker='*', s=60, color='green', zorder=20)

ax.set_ylim(top=7.5)
ax.set_xlabel("Interpolation")
ax.set_ylabel("Dot Product")
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

plt.savefig("./outputs/01_image_dot_product/interpolate_image_dot.pdf", bbox_inches='tight')

## For euclidean distance

In [ ]:
### From zeros to G
dists0 = []
for alpha in alphas:
    A = alpha*G #+ (1-alpha)*W
    d = np.linalg.norm((A-G).reshape(-1), ord=2)  ### calculating the euclidean_distance
    dists0.append(d)

### From G to ones
dists1 = []
for alpha in alphas:
    A = alpha*W + (1-alpha)*G
    d = np.linalg.norm((A-G).reshape(-1), ord=2)  ### calculating the euclidean_distance
    dists1.append(d)

In [ ]:
plt.plot(alphas, dists0, marker='o')
plt.plot(alphas+1, dists1, marker='o')

In [ ]:
## Plot the output with images

fig, ax = plt.subplots(1, 1, figsize=(6,5))

plt.plot(alphas, dists0, marker='o')
plt.plot(alphas+1, dists1, marker='o')

for idx_alpha, alpha in enumerate(alphas):
    if idx_alpha+1 == len(alphas):
        continue
    A = alpha*G #+ (1-alpha)*W
    d = np.linalg.norm((A-G).reshape(-1), ord=2)  ### calculating the euclidean_distance
    
    im = OffsetImage(A, norm=matplotlib.colors.Normalize(vmin=0, vmax=1), cmap="gray", zoom=3)
    ab = AnnotationBbox(im, (alpha, d+0.3), frameon=True, pad=0)
    ax.add_artist(ab)
    
for alpha in alphas:
    A = alpha*W + (1-alpha)*G
    d = np.linalg.norm((A-G).reshape(-1), ord=2)  ### calculating the euclidean_distance
    im = OffsetImage(A, norm=matplotlib.colors.Normalize(vmin=0, vmax=1), cmap="gray", zoom=3)
    ab = AnnotationBbox(im, (1+alpha, d+0.4), frameon=True, pad=0)
    ax.add_artist(ab)

ax.scatter(alphas[-1], dists0[-1], marker='*', s=100, color='green', zorder=20)
ax.set_ylim(top=4.55)
ax.set_xlabel("Interpolation")
ax.set_ylabel("Euclidean Distance")
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

plt.savefig("./outputs/01_image_dot_product/interpolate_image_distance.pdf", bbox_inches='tight')

## For angle / cos(theta)

In [ ]:
### From zeros to G
dists0 = []
for alpha in alphas:
    A = alpha*G #+ (1-alpha)*W
    u, v = A.reshape(-1), G.reshape(-1)
    d = np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(v)) ## calculating cosine angle
    if alpha == 0:
        d = 1
    d = np.clip(d, -1, 1)
    dists0.append(d)

### From G to ones
dists1 = []
for alpha in alphas:
    A = alpha*W + (1-alpha)*G
    u, v = A.reshape(-1), G.reshape(-1)
    d = np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(v)+1e-15) ## calculating cosine angle
    d = np.clip(d, -1, 1)
    dists1.append(d)

In [ ]:
plt.plot(alphas, dists0, marker='o')
plt.plot(alphas+1, dists1, marker='o')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,5))

for idx_alpha, alpha in enumerate(alphas):
    A = alpha*G #+ (1-alpha)*W
    d = dists0[idx_alpha]
    im = OffsetImage(A, norm=matplotlib.colors.Normalize(vmin=0, vmax=1), cmap="gray", zoom=3)
    ab = AnnotationBbox(im, (alpha, d+0.02), frameon=True, pad=0)
    ax.add_artist(ab)
    
for idx_alpha, alpha in enumerate(alphas):
    if idx_alpha == 0:
        continue
    A = alpha*W + (1-alpha)*G
    d = dists1[idx_alpha]
    im = OffsetImage(A, norm=matplotlib.colors.Normalize(vmin=0, vmax=1), cmap="gray", zoom=3)
    ab = AnnotationBbox(im, (1+alpha, d+0.033), frameon=True, pad=0)
    ax.add_artist(ab)

ax.scatter(alphas[-1], dists0[-1], marker='*', s=120, color='green', zorder=20)

plt.plot(alphas, dists0, marker='o')
plt.plot(alphas+1, dists1, marker='o')

ax.set_ylim(top=1.06)
ax.set_xlabel("Interpolation")
ax.set_ylabel("Cosine Angle")
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

plt.savefig("./outputs/01_image_dot_product/interpolate_image_angle.pdf", bbox_inches='tight')